<a href="https://colab.research.google.com/github/ishancoderr/Data_preparetion_for_research/blob/master/GAN_RESEARCH_001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [21]:
# Load the data from GitHub
data = pd.read_csv("https://raw.githubusercontent.com/ishancoderr/Data_preparetion_for_research/master/Lanslide_rainfall_data%20.csv")

In [22]:
data

,longitude,latitude,acc_rainfall,hours,Avg. Intensity mm/hr,landslide
0,6.519725,80.154021,316,7,45,yes
1,6.586835,80.160486,259,7,37,yes
2,6.093508,80.606877,119,4,30,yes
3,6.509504,80.231427,244,5,49,yes
4,6.260793,80.491387,237,6,40,yes
...,...,...,...,...,...,...
87,6.765405,80.868156,135,4,34,yes
88,6.965074,80.516389,316,7,45,yes
89,6.879253,80.442860,259,7,37,yes
90,6.848140,80.605650,119,4,30,yes


In [23]:
data['landslide'] = data['landslide'].replace({ 'yes': 1})

In [24]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
data[['acc_rainfall','hours'	,'Avg. Intensity mm/hr']] = scaler.fit_transform(data[['acc_rainfall','hours'	,'Avg. Intensity mm/hr']])

In [25]:
data

,longitude,latitude,acc_rainfall,hours,Avg. Intensity mm/hr,landslide
0,6.519725,80.154021,0.904762,0.8,0.689655,1
1,6.586835,80.160486,0.658009,0.8,0.413793,1
2,6.093508,80.606877,0.051948,0.2,0.172414,1
3,6.509504,80.231427,0.593074,0.4,0.827586,1
4,6.260793,80.491387,0.562771,0.6,0.517241,1
...,...,...,...,...,...,...
87,6.765405,80.868156,0.121212,0.2,0.310345,1
88,6.965074,80.516389,0.904762,0.8,0.689655,1
89,6.879253,80.442860,0.658009,0.8,0.413793,1
90,6.848140,80.605650,0.051948,0.2,0.172414,1


In [28]:
# Split the data into input and target variables
X = data[['acc_rainfall','hours'	,'Avg. Intensity mm/hr']].values

In [29]:
y=data['landslide']

In [30]:
# Define the generator network
def generator():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(64, activation='relu', input_shape=(None,3)))
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(32, activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    return model

In [31]:
# Define the discriminator network
def discriminator():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(64, activation='relu', input_shape=(None,1)))
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(32, activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    return model

In [32]:
# Compile the generator and discriminator
generator = generator()
discriminator = discriminator()

generator.compile(loss='mean_squared_error', optimizer='adam')
discriminator.compile(loss='mean_squared_error', optimizer='adam')

In [33]:
# Combine the generator and discriminator into a GAN
gan = tf.keras.Sequential([generator, discriminator])
gan.compile(loss='mean_squared_error', optimizer='adam')

In [34]:
# Train the GAN
for epoch in range(700):
    # Train the discriminator
    X_fake = generator.predict(X)
     

    y_fake = np.zeros(len(y))
    X_real = X
    y_real = y

    discriminator.trainable = True
    discriminator.train_on_batch(X_real, y_real)
    discriminator.train_on_batch(X_fake, y_fake)

    # Train the generator
    discriminator.trainable = False
    gan.train_on_batch(X, y_real)

3/3 [==============================] - 0s 3ms/step


3/3 [==============================] - 0s 5ms/step


In [35]:
print(discriminator.evaluate(X_real,y_real,verbose=0)) 
print(discriminator.evaluate(X_fake,y_fake, verbose=0)) 

0.029752153903245926
0.5931758880615234


In [40]:
# Define the input for the new data
user_input = np.array([[10,5,2]]) # example user input

user_input_scaled = scaler.fit_transform(user_input)
# Generate a prediction for the new data
prediction = generator.predict(user_input_scaled)
print(prediction)
# Convert the prediction to a binary output (0 or 1)
if prediction < 0.5:
    output = 0 # Generated_No
else:
    output = 1 # yes

# Print the output
print(output)

1/1 [==============================] - 0s 31ms/step
[[0.9938563]]
1
